imports

In [1]:
from tqdm.notebook import tqdm as tqdm
import os
import json
import pandas as pd

from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
from myhelpers import TrialStatistics

parameters

In [2]:
experimentsPath="/home/elhamod/HGNN/experiments/"
dataPath="/data/BGNN_data"
experimentName="biology_paper_dataset_differentModels"

cuda=6

file and changes

In [3]:
old_params_file = os.path.join(experimentsPath, experimentName, "params.json")

# Keep same order as ConfigParserWriter!
changes = {
    'numOfTrials': 3
}

prepare

In [4]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)

# populate a hash table of all experiments, indexed by trial hash
experiments = {}
for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
    num_of_models = experiment_params["numOfTrials"]
    for k in range(num_of_models):
        old_trialHash = TrialStatistics.getTrialName(experiment_params, k)
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        new_trialHash = TrialStatistics.getTrialName(experiment_params_new, k)
        experiments[old_trialHash] = {"trialHash": new_trialHash}

Fix data splits

In [5]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        # update experiment params
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        # fix data split names
        old_dataset_name = getDatasetName(config_parser.fixPaths(experiment_params))
        old_datasplitsPath = os.path.join(experimentsPath, experimentName, old_dataset_name)
        new_dataset_name = getDatasetName(config_parser.fixPaths(experiment_params_new))
        new_datasplitsPath = os.path.join(experimentsPath, experimentName, new_dataset_name)
        
        try:
            if os.path.exists(old_datasplitsPath):

                # update the folder names
                os.rename(old_datasplitsPath, new_datasplitsPath)
                print("datasplit", old_datasplitsPath, '->', new_datasplitsPath, "updated")

                # update the params.json
                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_datasplitsPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
                
            else:
                raise
                
        except:
            print("datasplit", old_dataset_name, "could not be fixed to", new_dataset_name)
            pass   
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['datasetName'] = new_dataset_name

        
        bar.update()

datasplit /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/datasplits/b4896471167c5599680115e51349929def797e403bead8f0fe554d79 -> /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/datasplits/b4896471167c5599680115e51349929def797e403bead8f0fe554d79 updated
json /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/datasplits/b4896471167c5599680115e51349929def797e403bead8f0fe554d79/params.json updated
datasplit /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/datasplits/b4896471167c5599680115e51349929def797e403bead8f0fe554d79 -> /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/datasplits/b4896471167c5599680115e51349929def797e403bead8f0fe554d79 updated
json /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/datasplits/b4896471167c5599680115e51349929def797e403bead8f0fe554d79/params.json updated
datasplit /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/dataspl

Fix models

In [6]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        num_of_models = experiment_params["numOfTrials"]
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for k in range(num_of_models):
            
                # Fix model names
                old_model_name = getModelName(experiment_params, k)
                old_modelPath = os.path.join(experimentsPath, experimentName, old_model_name)
                new_model_name = getModelName(experiment_params_new, k)
                new_modelPath = os.path.join(experimentsPath, experimentName, new_model_name)

                try:
                    if os.path.exists(old_modelPath):
                        os.rename(old_modelPath, new_modelPath)
                        print("model", old_modelPath, '->', new_modelPath, "updated")

                        j = json.dumps(experiment_params_new)
                        json_fileName = os.path.join(new_modelPath, 'params.json')
                        f = open(json_fileName,"w")        
                        f.write(j)
                        f.close()  
                        print("json", json_fileName, "updated")
                    else:
                        raise
                    
                except:
                    print("model", old_model_name, "could not be fixed to", new_model_name)
                    pass  
                
                

                bar2.update()
                
                experiments[TrialStatistics.getTrialName(experiment_params, k)]['modelName'] = new_model_name

        bar.update()

model /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/models/907fb6a09efd800575291e3ec0a59c6773c600417bf1dee04b571b9f -> /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/models/825aa4c30a7ace1285f54f2f26af4f9702b5fec2a2f7edf3b35666f7 updated
json /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/models/825aa4c30a7ace1285f54f2f26af4f9702b5fec2a2f7edf3b35666f7/params.json updated
model /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/models/c848e37ed5229c2d3e651dbb81b74cda5b4806aaccb2da9587216119 -> /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/models/8f4c827179c7284f264c3da790cbe741cb17f91416b995aafedd7720 updated
json /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/models/8f4c827179c7284f264c3da790cbe741cb17f91416b995aafedd7720/params.json updated
model /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/models/8bc778169c754715001221b39327c06d4033

model models/d014becdcaaa06e69d07ed7cd289b89f475c1bece84b45c5992ab320 could not be fixed to models/1f473114ca43b4ceb1a2ecb70ead11478584241e97458fc74d6694e0
model models/de1ad45a560caea97ae654abf0e8c4b49677e45eeb38eea6b3951c67 could not be fixed to models/a17fa195789d6a3dd1c963087bd980c9ff352075349b8427af918b73
model models/d2807d5aba0574f8fd644f199693a91f6ac76d6962a806c2b1dd5509 could not be fixed to models/3645aac312be7c37e7efa6c147262f308eeafd88d1361603f04bb34e
model models/e43c45998663847ceedc406a8f7bbececb54deb3fda4146035b9c83e could not be fixed to models/0764ecca7cb57827cb9b7d129721b4cdc4d8caaa352282268c4b6282
model models/c4f7c78d6666d8b7c62b580ea05e9a23b2ad9698212724dd5cc967f4 could not be fixed to models/c16fada3ecd65b6dc496f00c77a06db9c738a372cd4206bc0af21cd5



model models/03b2491251697b0ddf9cbc33efed513e7650235554adc1e942d52767 could not be fixed to models/6d98bcb1f2bda1058dd8c8bc28366920235a96a2c62a152e317e19a0
model models/1497b1db20d84585f556077f836d8201e3d9ef650cc698384d6a8c1d could not be fixed to models/975c676dc16f1dbb79009478a3855f9a2d36216c347d8f7b8bfe7549
model models/f43b5dfdc59f20edc17ba7559c98b308e5d6368ba55eb28bf879d3cf could not be fixed to models/09337aa6e614a87569e4e0ad7041c8b1da8eab80732b5f6b5d1ed3e2
model models/3ee66bf3b986d94be69a5a88b6ba1ab551ccb1f4c3515061aa0fe525 could not be fixed to models/c3fe09a1d5e4aca5e33f466ccb1b4e8d764a93510d70e5499d231562
model models/2f74b54d3df48c2c4505e5b9410e09ce14c618f7e6a4b6f45930831e could not be fixed to models/c46de6793e59be53423dbffa3e166afa09729cdfc40dcf16867714c8



model models/e9274f43a9c6d1edb6425c12b87f17dae4aa9eff6fa5f61f0ed145aa could not be fixed to models/5735cea1d437d01b5ae9ab36263859720f3fcde6f1d7e1299bc1f470
model models/68d2b9d166dfdc7ee01b4541f597c8b3e78aa0c9f116134c6d803754 could not be fixed to models/d7ef4d63e71973c6aad10eaf7fd32c1c88ad2e96a6f89471bf46e4c2
model models/79768b7a8f25e4ffaaa337d292c05522c445d2a37b59f12a46831425 could not be fixed to models/080895e702964b84b97c64d4429dc024850eb9541f2afbe1d5452768
model models/e35ef96c9aa5cacd0e2618271dd54d3ed1be22e5d3267466f44b23d9 could not be fixed to models/60e192dc562e5736f8640c0ba0cf3ed756ebe9c71c730d74adf91816
model models/917779be400e44d87826660e83562b2e7d76373f2fb6edb649dad309 could not be fixed to models/e6926a93a4f6a79f86493711245bb49512c2c28142d9264ec9fcafd2




Fix results directory

In [7]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        old_experimentHash = TrialStatistics.getTrialName(experiment_params)
        old_experimentPath = os.path.join(experimentsPath, experimentName, "results", old_experimentHash)
        new_experimentHash = TrialStatistics.getTrialName(experiment_params_new)
        new_experimentPath = os.path.join(experimentsPath, experimentName, "results", new_experimentHash)
        
        try:
            if os.path.exists(old_experimentPath):

                os.rename(old_experimentPath, new_experimentPath)
                print("result", old_experimentPath, '->', new_experimentPath, "updated")

                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_experimentPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
            else:
                raise
                
        except:
            print("result", old_experimentHash, "could not be fixed to",new_experimentHash)
            pass 
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['experimentHash'] = new_experimentHash
        
        bar.update()

result dc9cdf5d84792686be05ad04199b4691281d7c64533198ad5baa2e78 could not be fixed to 091351b87556c0d52115d41316e73819c38ea32f4790877f20fc5137
result 47e062f28bdd7dc8768626a462508fca6f4a73b1693594d1c5b5d708 could not be fixed to a8fbd8e6a0b45aae4ef6c68ed38dee2c5d46dbafa26e0eb569d43930
result 39e82b1034de354855512342447991ea092e1c4a3bb9ba59613939c5 could not be fixed to 8c4db863b698c3d66823184d97b42eb99dd4b88b109c56df04fa7ff5
result 109e15bd14b51286e5dcf8dd2630b2fba01de4f027517c124454a24f could not be fixed to ba1048373df76e378c958f977a494db049271e4b70421db38c5d99d7



Fix experiments file

In [8]:
# open experiments.csv file
experimentsFileNameAndPath=os.path.join(experimentsPath, "experiments.csv")
experiments_file= pd.read_csv(experimentsFileNameAndPath)

# For all experiments
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        numOfTrials = experiment_params["numOfTrials"]
        trialHash_0 = TrialStatistics.getTrialName(experiment_params, 0) # used for keys who don't change for different trials
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        
        # For all trials
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for j in range(numOfTrials):
                
                trialHash = TrialStatistics.getTrialName(experiment_params, j)
                try:
                    # If old entry doesn't exist, nothing to update
                    record_exists = not (experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName]).empty
                    if record_exists:
                        
                        # drop old row and caluclate new row
                        new_trialHash = experiments[trialHash]["trialHash"]
                        experiments_file.drop(experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 
                        row_information = {
                            'experimentName': experimentName,
                            'modelName': experiments[trialHash]["modelName"],
                            'datasetName': experiments[trialHash_0]["datasetName"],
                            'experimentHash': experiments[trialHash_0]["experimentHash"],
                            'trialHash': new_trialHash
                        }
                        row_information = {**row_information, **experiment_params_new} 

                        # error handling to avoid duplicates
                        record_exists = not (experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName]).empty
                        if record_exists:
                            experiments_file.drop(experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 

                        # update with new entry    
                        experiments_file = experiments_file.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
                        print("trialHash", trialHash, "->", new_trialHash, "updated")
                except:
                    print("trialHash", trialHash, "could not be updated!")
                    pass
                bar2.update()
        bar.update()

# resave file
experiments_file.to_csv(experimentsFileNameAndPath, header=True, index=False)

trialHash 907fb6a09efd800575291e3ec0a59c6773c600417bf1dee04b571b9f -> 825aa4c30a7ace1285f54f2f26af4f9702b5fec2a2f7edf3b35666f7 updated
trialHash c848e37ed5229c2d3e651dbb81b74cda5b4806aaccb2da9587216119 -> 8f4c827179c7284f264c3da790cbe741cb17f91416b995aafedd7720 updated
trialHash 8bc778169c754715001221b39327c06d40333454eeebb19bb166881d -> a4dba7925ad2409f8467537935803f353cdecd311cbfba8267c00bbd updated



/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Fix params file

In [9]:
experiment_list = []
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_list.append(experiment_params_new)
        bar.update()


j = json.dumps({"experimentList": experiment_list})
f = open(old_params_file,"w")        
f.write(j)
f.close()  
print("json", old_params_file, "updated")


json /home/elhamod/HGNN/experiments/biology_paper_dataset_differentModels/params.json updated
